# Bledin's compositional truthmaker semantics

## Fineian truthmaker semantics

In [1]:
{-# LANGUAGE OverloadedLists, FlexibleContexts, DeriveTraversable, ParallelListComp #-}

import Data.Semilattice.Join
import qualified Data.Map as M
import qualified Data.Set as S
import qualified Data.IntSet as IS
import Control.Monad (replicateM)

type S = IS.IntSet

compare (LessThan ([1] :: S)) (LessThan [1,2])

LT

## Syntax

In [2]:
data L a = Atom a | Not (L a) | (L a) :&: (L a) | (L a) :|: (L a) deriving (Foldable, Traversable, Functor)

type Expr = L Char

type Ext = Bool -> S.Set S

variables :: Expr -> S.Set Char
variables = foldr S.insert S.empty

-- Note that (Bool -> a) is isomorphic to (a,a)
type Assignment = Char -> Bool -> S.Set S

test1 = Atom 'a' :&: Atom 'b'

variables test1

fromList "ab"

## Semantics

The basic evaluation algorithm for Fine's sentential truthmaker semantics.

In [28]:
eval :: Assignment -> Expr -> Bool -> S.Set S
eval g (Atom a) = g a
eval g (Not expr) = eval g expr . not -- this flips the positive to the negative extension
eval g (lconj :&: rconj) = \t ->
        if t
            then S.fromList [s \/ s' | s <- S.toList $ eval g lconj t, s' <- S.toList $ eval g rconj t]
            else eval g lconj t `S.union` eval g rconj t
eval g (lconj :|: rconj) = \t ->
        if t
            then eval g lconj t `S.union` eval g rconj t
            else S.fromList [s \/ s' | s <- S.toList $ eval g lconj t, s' <- S.toList $ eval g rconj t]


Line 6: Reduce duplication
Found:
s <- S.toList $ eval g lconj t
s' <- S.toList $ eval g rconj t
s \/ s'

Why not:
Combine with -:11:40-69